In [94]:
import pandas as pd
import numpy as np

from skrub import TableVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

import holidays

from sklearn.metrics import mean_squared_error


In [95]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")
external_data_cleaned = pd.read_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/external_data_cleaned.csv")

In [96]:
# we convert 'date' column to datetime in all datasets
external_data_cleaned['date'] = pd.to_datetime(external_data_cleaned['date'])
df_train['date'] = pd.to_datetime(df_train['date']).astype('datetime64[ns]')
df_test['date'] = pd.to_datetime(df_test['date']).astype('datetime64[ns]')

# Sort datasets by date
external_data_cleaned.sort_values('date', inplace=True)
df_train.sort_values('date', inplace=True)
df_test.sort_values('date', inplace=True)

# we merge the data together  by performing the nearest match in terms of date :
training_set_merged = pd.merge_asof(df_train, external_data_cleaned, on='date', direction='nearest')
testing_set_merged = pd.merge_asof(df_test, external_data_cleaned, on='date', direction='nearest')

In [97]:
# Extract the date feature on different time scales :
fr_holidays = holidays.France()

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Add a feature to indicate if the day is a holiday in France
    X["is_holiday"] = X["date"].apply(lambda d: 1 if d in fr_holidays else 0)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

training_set_merged = _encode_dates(training_set_merged)
testing_set_merged = _encode_dates(testing_set_merged)


In [98]:
training_set_merged.head()

,counter_id,counter_name,site_id,site_name,bike_count,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,...,weather_condition_score,precipitation_score,snow_score,year,month,day,weekday,hour,is_weekend,is_holiday
0,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,1.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,...,-0.479885,-0.392827,0.04679,2020,9,1,1,1,0,0
1,100056226-103056226,Face au 8 avenue de la porte de Charenton SE-NO,100056226,Face au 8 avenue de la porte de Charenton,1.0,2019-11-01,"48.830331,2.400551",Y2H19070370,48.830331,2.400551,...,-0.479885,-0.392827,0.04679,2020,9,1,1,1,0,0
2,100047545-104047545,Face 104 rue d'Aubervilliers S-N,100047545,Face 104 rue d'Aubervilliers,1.0,2018-11-29,"48.890457,2.368852",Y2H18086321,48.890457,2.368852,...,-0.479885,-0.392827,0.04679,2020,9,1,1,1,0,0
3,100060178-102060178,90 Rue De Sèvres NE-SO,100060178,90 Rue De Sèvres,21.0,2020-07-22,"48.84638,2.31529",Y2H20052705,48.846380,2.315290,...,-0.479885,-0.392827,0.04679,2020,9,1,1,1,0,0
4,100056327-103056327,Face au 4 avenue de la porte de Bagnolet E-O,100056327,Face au 4 avenue de la porte de Bagnolet,2.0,2019-11-06,"48.86461,2.40969",Y2H19070372,48.864610,2.409690,...,-0.479885,-0.392827,0.04679,2020,9,1,1,1,0,0


In [99]:
training_set_merged.columns

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'counter_installation_date', 'coordinates', 'counter_technical_id',
       'latitude', 'longitude', 'log_bike_count', 'wind_speed', 'temperature',
       'humidty', 'visibility', 'etat_sol', 'pressure_score',
       'cloudiness_score', 'weather_condition_score', 'precipitation_score',
       'snow_score', 'year', 'month', 'day', 'weekday', 'hour', 'is_weekend',
       'is_holiday'],
      dtype='object')

In [100]:
training_set_merged = training_set_merged.drop(columns=['wind_speed', 'pressure_score',
       'cloudiness_score', 'humidty', 'visibility', 'etat_sol', 'pressure_score',
       'cloudiness_score', 'weather_condition_score', 'precipitation_score',
       'snow_score', ])

testing_set_merged = testing_set_merged.drop(columns=['wind_speed', 'pressure_score',
       'cloudiness_score', 'humidty', 'visibility', 'etat_sol', 'pressure_score',
       'cloudiness_score', 'weather_condition_score', 'precipitation_score',
       'snow_score', ])

In [101]:
training_set_merged.columns

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'counter_installation_date', 'coordinates', 'counter_technical_id',
       'latitude', 'longitude', 'log_bike_count', 'temperature', 'year',
       'month', 'day', 'weekday', 'hour', 'is_weekend', 'is_holiday'],
      dtype='object')

In [102]:
# Define bins and labels for temperature categories in Kelvin
bins = [-float('inf'), 278.15, 283, 298, 308.15, float('inf')]  # Updated Kelvin thresholds
labels = ['very_cold', 'cold', 'moderate', 'warm', 'very_hot']

# Create a new categorical feature for temperature
training_set_merged['temp_category'] = pd.cut(training_set_merged['temperature'], bins=bins, labels=labels)
testing_set_merged['temp_category'] = pd.cut(testing_set_merged['temperature'], bins=bins, labels=labels)

# One-hot encode the categories for the model
training_set_merged = pd.get_dummies(training_set_merged, columns=['temp_category'], drop_first=True)
testing_set_merged = pd.get_dummies(testing_set_merged, columns=['temp_category'], drop_first=True)

# remove temperature column :
training_set_merged = training_set_merged.drop(columns=['temperature'])
testing_set_merged = testing_set_merged.drop(columns=['temperature'])

In [103]:
training_set_merged.head(30)

,counter_id,counter_name,site_id,site_name,bike_count,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,...,month,day,weekday,hour,is_weekend,is_holiday,temp_category_cold,temp_category_moderate,temp_category_warm,temp_category_very_hot
0,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,1.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,...,9,1,1,1,0,0,False,False,False,False
1,100056226-103056226,Face au 8 avenue de la porte de Charenton SE-NO,100056226,Face au 8 avenue de la porte de Charenton,1.0,2019-11-01,"48.830331,2.400551",Y2H19070370,48.830331,2.400551,...,9,1,1,1,0,0,False,False,False,False
2,100047545-104047545,Face 104 rue d'Aubervilliers S-N,100047545,Face 104 rue d'Aubervilliers,1.0,2018-11-29,"48.890457,2.368852",Y2H18086321,48.890457,2.368852,...,9,1,1,1,0,0,False,False,False,False
3,100060178-102060178,90 Rue De Sèvres NE-SO,100060178,90 Rue De Sèvres,21.0,2020-07-22,"48.84638,2.31529",Y2H20052705,48.846380,2.315290,...,9,1,1,1,0,0,False,False,False,False
4,100056327-103056327,Face au 4 avenue de la porte de Bagnolet E-O,100056327,Face au 4 avenue de la porte de Bagnolet,2.0,2019-11-06,"48.86461,2.40969",Y2H19070372,48.864610,2.409690,...,9,1,1,1,0,0,False,False,False,False
5,100047545-103047545,Face 104 rue d'Aubervilliers N-S,100047545,Face 104 rue d'Aubervilliers,3.0,2018-11-29,"48.890457,2.368852",Y2H18086321,48.890457,2.368852,...,9,1,1,1,0,0,False,False,False,False
6,100056336-105056336,27 quai de la Tournelle SE-NO,100056336,27 quai de la Tournelle,3.0,2019-11-14,"48.85013,2.35423",Y2H19070383,48.850130,2.354230,...,9,1,1,1,0,0,False,False,False,False
7,100047546-104047546,Face au 70 quai de Bercy N-S,100047546,Face au 70 quai de Bercy,0.0,2018-11-30,"48.829523,2.38699",Y2H20022165,48.829523,2.386990,...,9,1,1,1,0,0,False,False,False,False
8,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,3.0,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233,...,9,1,1,1,0,0,False,False,False,False
9,100060178-101060178,90 Rue De Sèvres SO-NE,100060178,90 Rue De Sèvres,0.0,2020-07-22,"48.84638,2.31529",Y2H20052705,48.846380,2.315290,...,9,1,1,1,0,0,False,False,False,False


In [104]:
training_set_merged.nunique()

counter_id                    56
counter_name                  56
site_id                       30
site_name                     30
bike_count                   998
counter_installation_date     22
coordinates                   30
counter_technical_id          30
latitude                      30
longitude                     30
log_bike_count               998
year                           2
month                         12
day                           31
weekday                        7
hour                          24
is_weekend                     2
is_holiday                     2
temp_category_cold             1
temp_category_moderate         1
temp_category_warm             1
temp_category_very_hot         1
dtype: int64

In [61]:
X_train = training_set_merged.drop(columns=["bike_count", "log_bike_count"])
y_train = training_set_merged["log_bike_count"]

X_test = testing_set_merged.copy()

In [62]:
# Preprocessing pipeline
pipeline = Pipeline(
    steps=[
        ('preprocessor', TableVectorizer()),
        ('model', xgb.XGBRegressor()),
    ]
)

In [63]:
# Fit Pipeline to Training Data
pipeline.fit(X_train, y_train)

# Make Predictions on Test Data
y_predictions = pipeline.predict(X_test)


In [64]:
print(y_predictions)

[0.16022734 1.2152132  1.3688272  ... 2.715215   2.9325204  2.953472  ]


In [65]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_option_2_vsimple_weather_preprocessed.csv", index=False)

In [66]:
# We can print the RMSE on the training data :
y_train_predictions = pipeline.predict(X_train)
rmse_train = mean_squared_error(y_train, y_train_predictions, squared=False)
print(f"Training RMSE: {rmse_train}")

Training RMSE: 0.40746825662304575


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [67]:
# code to get feature importance :


# Step 1: Extract the preprocessor and feature names
# Retrieve the preprocessor from the pipeline
preprocessor = pipeline.named_steps['preprocessor']

# Get the feature names after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Step 2: Extract the trained XGBoost model and feature importance
xgb_model = pipeline.named_steps['model']

# Get feature importances from the trained XGBoost model
feature_importance = xgb_model.feature_importances_

# Step 3: Combine feature names and importance scores into a DataFrame
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Sort features by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display top features
print("Top Features by Importance:")
importance_df.head(40)

Top Features by Importance:


,Feature,Importance
47,"counter_name: invalides, des, de",0.106246
16,"counter_id: 100056331, 104056331, 103056331",0.079822
39,"counter_name: sébastopol, 73, de",0.072617
41,"counter_name: cours, reine, totem",0.065688
93,counter_installation_date_day,0.064716
14,"counter_id: 100050876, 103050876, 104050876",0.052867
6,"counter_id: 102007049, 101007049, 100007049",0.051410
162,hour,0.045962
60,site_id,0.044098
53,"counter_name: tournelle, 27, no",0.035823
